In [1]:
import pandas as pd

In [2]:
carnegie_full = pd.read_csv(r'C:\\Users\\dmcad\\Desktop\\DDA9\\Capstone\\carnegie_basic.csv')
finance_full = pd.read_csv(r'C:\\Users\\dmcad\\Desktop\\DDA9\\Capstone\\ef2021d.csv')
emi_full = pd.read_excel(r'C:\\Users\\dmcad\\Desktop\\DDA9\\Capstone\\emi.xlsx')
character_full = pd.read_csv(r'C:\\\\Users\\\\dmcad\\\\Desktop\\\\DDA9\\\\Capstone\\\\hd2021.csv', encoding='ISO-8859-1')
socmo_full = pd.read_csv(r'C:\\Users\\dmcad\\Desktop\\DDA9\\Capstone\\smi22.csv')

In [3]:
# Renaming columns
emi_full.rename(columns={
    'Institution': 'school', 
    'Control': 'control', 
    'Economic Mobility Index': 'pell_emi', 
    'Low-income PEP Percentile Rank': 'pell_pep',
    'Price-to-Earnings Premium: Years to Pay Down Total Net Cost': 'gen_pep'}, inplace=True)
character_full.rename(columns={
    'UNITID': 'unitid', 
    'INSTNM': 'match_check', 
    'STABBR': 'state', 
    'ZIP': 'zip', 
    'LONGITUD': 'long', 
    'LATITUDE': 'lat', 
    'NPRICURL': 'calc_url', 
    'HBCU': 'hbcu', 
    'LOCALE': 'setting', 
    'C21BASIC': 'c21basic', 
    'C21IPUG': 'c21ipug'}, inplace=True)
finance_full.rename(columns={'RET_PCF': 'retention', 
                             'GRCOHRT': 'class_size'}, inplace=True)
socmo_full.rename(columns={
    'Institution': 'school', 
    'Average Debt': 'coa', 
    'Endowment (in M)': 'endowment'}, inplace=True)

In [4]:
# Merging dataframes; has match_check column
base = pd.merge(emi_full[['school', 'control', 'gen_pep', 'pell_emi', 'pell_pep']], character_full[['unitid', 'match_check', 'state', 'zip', 'long', 'lat', 'calc_url', 'hbcu', 'setting', 'c21basic', 'c21ipug']], left_on='school', right_on='match_check', how='inner')
base = pd.merge(base, finance_full[['UNITID', 'retention', 'class_size']], left_on='unitid', right_on='UNITID', how='inner')
base = pd.merge(base, socmo_full[['school', 'coa', 'endowment']], on='school', how='inner')

In [5]:
# Keeping only necessary columns
base = base[['school', 
             'control',
             'class_size', 
             'gen_pep', 
             'pell_emi', 
             'pell_pep', 
             'unitid', 
             'state', 
             'zip', 
             'long', 
             'lat', 
             'calc_url', 
             'hbcu', 
             'setting', 
             'c21basic', 
             'c21ipug', 
             'retention', 
             'coa', 
             'endowment']]

In [7]:
finaid_full = pd.read_csv(r'C:\\Users\\dmcad\\Desktop\\DDA9\\Capstone\\sfa2021.csv')

In [8]:
# Add financial aid info and income comparisons
base = pd.merge(base, finaid_full[[
    "UNITID",
    "GIS4N12",
    "GIS4A12",
    "GIS4N22",
    "GIS4A22",
    "GIS4N32",
    "GIS4A32",
    "GIS4N42",
    "GIS4A42",
    "GIS4N52",
    "GIS4A52"
]], left_on="unitid", right_on="UNITID", how="inner")

In [9]:
base = base.rename(columns={
    "GIS4N12": "pell_count",
    "GIS4A12": "pell_aid",
    "GIS4N22": "inc1_count",
    "GIS4A22": "inc1_aid",
    "GIS4N32": "inc2_count",
    "GIS4A32": "inc2_aid",
    "GIS4N42": "inc3_count",
    "GIS4A42": "inc3_aid",
    "GIS4N52": "inc4_count",
    "GIS4A52": "inc4_aid"
})

In [10]:
#Calculate new EMIs
base['inc1_emi'] = (base['inc1_count'] / base['class_size']) * base['gen_pep']
base['inc2_emi'] = (base['inc2_count'] / base['class_size']) * base['gen_pep']
base['inc3_emi'] = (base['inc3_count'] / base['class_size']) * base['gen_pep']
base['inc4_emi'] = (base['inc4_count'] / base['class_size']) * base['gen_pep']

In [11]:
#Min-max normalization by state
def normalize(data):
    return (data - data.min()) / (data.max() - data.min()) * 100

In [12]:
grouped = base.groupby('state')[['inc1_emi', 'inc2_emi', 'inc3_emi', 'inc4_emi', 'pell_emi']]
base[['inc1_emi_norm', 'inc2_emi_norm', 'inc3_emi_norm', 'inc4_emi_norm', 'pell_emi_norm']] = grouped.transform(normalize)

In [16]:
base.head()

,school,control,class_size,gen_pep,pell_emi,pell_pep,unitid,state,zip,long,...,inc4_aid,inc1_emi,inc2_emi,inc3_emi,inc4_emi,inc1_emi_norm,inc2_emi_norm,inc3_emi_norm,inc4_emi_norm,pell_emi_norm
0,California State University-Los Angeles,Public,3895.0,0.425166,0.666779,0.980,110592,CA,90032-8506,-118.168965,...,1579.0,0.085906,0.051631,0.013754,0.010697,1.239029,0.000000,0.0,5.204874,100.000000
1,Florida International University,Public,2872.0,1.028683,0.447354,0.929,133951,FL,33199,-80.373928,...,6724.0,0.135391,0.093126,0.055159,0.097424,3.902008,0.000000,0.0,7.435486,100.000000
2,Fayetteville State University,Public,548.0,2.736935,0.400128,0.718,198543,NC,28301-4298,-78.891696,...,2285.0,0.529407,0.284681,0.149832,0.069922,100.000000,78.421245,0.0,0.000000,100.000000
3,Florida Agricultural and Mechanical University,Public,1139.0,3.494540,0.369889,0.611,133650,FL,32307,-84.290226,...,5697.0,0.454075,0.395782,0.260787,0.312944,100.000000,100.000000,100.0,100.000000,82.249253
4,Albion College,"Private, non-profit",468.0,1.631216,0.365916,0.867,168546,MI,49224,-84.745639,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.000000


In [15]:
base.to_csv('college_wishlist.csv', index=False)